In [ ]:
import numpy as np
import os
from sklearn.semi_supervised import LabelSpreading, LabelPropagation
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2, f_classif, mutual_info_classif
import time
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression, LassoCV, RandomizedLasso
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold #, GridSearchCV, cross_val_score

%matplotlib inline 
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib
from mpl_toolkits.mplot3d import Axes3D #, axes3d
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import Pipeline
import itertools


import operator
# import winsound



In [ ]:
datapath = 'data/'
XYfile = datapath + 'newfeatures_1024_1_10_10000_XY.npz'
Xpcafile = datapath + 'newfeatures_1024_1_10_10000_dimred.npz'
Xtestsds = datapath + 'newfeatures_1024_1_10_10000_testds.npz'
############# GATHERING into one complete array
#X = np.load(XYfile)['X']
with np.load(XYfile, encoding = 'latin1') as inpf:
    X = inpf['X']
    Y = inpf['Y']
# with np.load(Xpcafile, encoding = 'latin1') as inpf:
#     X = inpf['Xpca']
#     Y = inpf['Y']
#Y = np.load(XYfile)['Y']
# with np.load(Xtestsds, encoding = 'latin1') as inpf:
#     X = inpf['ds']


print ('gathered features: ', X[0].shape, Y[0].shape, X[1].shape, Y[1].shape, X[2].shape, Y[2].shape, np.sum(Y[2]))

In [ ]:
def make_pipe(clf,sca,order):
# order = 1 : first perform feature selection and then apply PCA
# order = 0 : first apply PCA and then reduce the transformed features
    if order:
        pipeline = Pipeline([('scaler', sca),
                    ('feature_selection', SelectKBest(mutual_info_classif,nfeat)),
                    ('decomp', PCA(ncomp)),        
                    ('classifier', clf) ])
    else:
        pipeline = Pipeline([('scaler', sca),
                    ('decomp', PCA(ncomp)),                 
                    ('feature_selection', SelectKBest(mutual_info_classif,nfeat)),        
                    ('classifier', clf) ])
    return pipeline

In [ ]:
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
#     """
#     This function prints and plots the confusion matrix.
#     Normalization can be applied by setting `normalize=True`.
#     """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

   
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("Normalized confusion matrix")
   
    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
# cnf_matrix = confusion_matrix(y_test, y_pred)
# np.set_printoptions(precision=2)

In [ ]:
# %%time
x_tot = np.copy(X[1]) ; y_tot = np.copy(Y[1])
x_tot = x_tot[0: :20] ; y_tot = y_tot[0: :20]
#Grid search for param tuning in SVM()
cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = state)

grid_pip = make_pipe(clf = SVC(), sca = StandardScaler(), order = 1)
params = {'feature_selection__k': (1000,500,100),
         'decomp__n_components': (100,50),
          'classifier__gamma' : ('auto',0.02, 0.2, 2),
          'classifier__C': (1,10,100),
         }
grid_search = GridSearchCV(grid_pip, params, verbose = 1, cv = cv)
grid_search.fit(x_tot, y_tot)
print("------ Grid search cv results for SVM ------")
print("Best score: %0.4f" %grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(params.keys()):
     print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
# winsound.Beep(400,900)